In [532]:
import json, random
import numpy as np
from pprint import pprint
from collections import Counter, defaultdict
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import pickle
import copy
import spacy
from spacy import displacy
np.set_printoptions(precision=4)

In [2]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_J.json", "r"))
print(len(txt_dataset))

7921


In [506]:
# Randomly look at positive facts and the wikipedia pages they come from
analyze_ids = random.sample(range(7921), 15)
print(analyze_ids)
for i in analyze_ids:
    k = str(i)
    print('Q: ', txt_dataset[k]['Q'])
    print('A: ', txt_dataset[k]['A'])
    for f in txt_dataset[k]['SupportingFacts']:
        print(f['fact'])
        print(f['url'])
    print()

[4330, 982, 5834, 6850, 3781, 697, 3929, 2376, 5909, 4968, 5537, 1581, 6812, 6693, 2102]
Q:  Did Nickelodeon or Netflix premiere Winx episodes first?
A:  Nickelodeon was first as it premiered the Winx Club series in 2012, whereas Netflix premiered the Winx Saga in 2021.
Its television premiere was on 11 March 2012 on Nickelodeon in the United States. The plot takes place after the events of the first three seasons. In 2007, production began on a sequel to The Secret of the Lost Kingdom, before the fourth season had been written. It was released in Italy on 29 October 2010.
https://en.wikipedia.org/wiki/List_of_Winx_Club_episodes
The six-episode first season debuted on Netflix on 22 January 2021 to mixed reviews. In February 2021, the series was renewed for a second season. Bloom, a fairy with fire powers, enrolls at a magical boarding school in the Otherworld called Alfea College.
https://en.wikipedia.org/wiki/Fate:_The_Winx_Saga

Q:  What is a concept that is centrally emphasized by b

Step 1: Question --> Identify salient spans

Step 2: Spans --> Wiki pages

Step 3: Wikipages --> html parsing --> paragraphs, images & their captions

Step 4: Choose paragraphs with high overlap with the question **without** giving away the core answer info

#### Step 1: Question --> Identify salient spans

In [526]:
nlp = spacy.load('en_core_web_sm')

In [527]:
def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
    else: print('No named entities found.')

In [673]:
print(wikipedia.search("Incan amaru"))

['Tupac Shakur', 'Inca Empire', 'Rebellion of Túpac Amaru II', 'Túpac Amaru II', 'Túpac Amaru', 'Spanish conquest of the Inca Empire', 'Neo-Inca State', 'Aramu Muru', 'Tupac (name)', 'Pachacuti']


In [585]:
print(wikipedia.search("Which partner in the Byzantine–Venetian Treaty of 1082 had the statues of a pair of armored late Roman emperors embracing each other last"))

[]


In [561]:
ids = random.sample(range(7921), 5)
for i in ids:
    k = str(i)
    print(k)
    Q = txt_dataset[k]['Q']
    print('Q: ', Q)
    print('A: ', txt_dataset[k]['A'])
    for f in txt_dataset[k]['SupportingFacts']:
        print(f['fact'])
        print(f['url'])
    print("\nEntities:")
    doc = nlp(Q)
    show_ents(doc)
    if doc.ents: displacy.render(doc, style='ent', jupyter=True)
    for e in doc.ents:
        print(wikipedia.search(e.text))
    print(" ---------------------------------------------------------------------------- \n")

5114
Q:  How much higher did Take That & Party peak on the UK Albums Chart than Having a Party peaked on the Billboard Hot 100?
A:  Take That & Party peaked 15 spots spots higher on it's respective chart.
Released on 17 August 1992 on RCA Records, it reached number two on the UK Albums Chart and stayed in the UK Top 75 album chart for 73 weeks (one year, five months and one week). It was their only album not to reach number one until Wonderland peaked at number two in 2017.
https://en.wikipedia.org/wiki/Take_That_%26_Party
The song peaked at number four on Billboard 's Hot R&B Sides chart, and also charted at number 17 on the Billboard Hot 100 . "Having a Party", like its A-side, "Bring It On Home to Me", was written while Cooke was on tour for Henry Wynn.
https://en.wikipedia.org/wiki/Having_a_Party_(song)

Entities:
Take That & Party - 20 - 37 - ORG - Companies, agencies, institutions, etc.
UK - 50 - 52 - GPE - Countries, cities, states
Party - 80 - 85 - ORG - Companies, agencies, in

['Take That & Party', 'Take That', 'List of songs recorded by Take That', 'Take That discography', 'Wonderland (Take That album)', 'Greatest Hits (Take That album)', 'Could It Be Magic', 'Take That Presents: The Circus Live', 'III (Take That album)', 'Jason Orange']
['United Kingdom', 'Countries of the United Kingdom', '.uk', '2016 United Kingdom European Union membership referendum', 'List of political parties in the United Kingdom', 'UK garage', 'Postcodes in the United Kingdom', 'Sky UK', 'Prime Minister of the United Kingdom', 'UK Biobank']
['Party', 'Political party', 'Labour Party (UK)', 'Republican Party (United States)', 'Political parties in the United States', 'Social Democratic Party of Germany', 'One-party state', 'Democratic Party (United States)', 'Black Panther Party', 'National Party of Australia']
['100', '100 metres', '100%', 'Nasdaq-100', '$100', 'The 100', 'Time 100', 'Billboard Hot 100', "List of men's footballers with 100 or more international caps", '$100 hamburg

/home/yingshac/miniconda3/envs/py37/lib/python3.7/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


 ---------------------------------------------------------------------------- 

515
Q:  What cardinal region of Hartford, Connecticut do the American soccer team Hartford City FC and University of Hartford's Hartford Hawks lie in?
A:  In West Hartford.
The Hartford Hawks are the NCAA Division I athletic teams of the University of Hartford, located in West Hartford, Connecticut.
https://en.wikipedia.org/wiki/Hartford_Hawks_softball
Hartford City FC is an American soccer team based in West Hartford, Connecticut. The team plays in the National Premier Soccer Leagues' Northeast Region, North Atlantic Conference.
https://en.wikipedia.org/wiki/Hartford_City_FC

Entities:
Hartford - 24 - 32 - GPE - Countries, cities, states
Connecticut - 34 - 45 - GPE - Countries, cities, states
American - 53 - 61 - NORP - Nationalities or religious or political groups
Hartford City FC - 74 - 90 - GPE - Countries, cities, states
University of Hartford's - 95 - 119 - ORG - Companies, agencies, institutions, et

['Hartford, Connecticut', 'The Hartford', 'Hartford Whalers', 'Travelers Championship', 'Dee Hartford', 'Greater Hartford', 'Asa Hartford', 'Hartford Athletic', 'Hartford Courant', 'Eden Hartford']
['Connecticut', 'The Haunting in Connecticut', 'Hartford, Connecticut', 'University of Connecticut', 'Connecticut River', 'List of municipalities in Connecticut', 'Bridgeport, Connecticut', 'Stamford, Connecticut', 'Sandy Hook Elementary School shooting', 'Connecticut Lakes']
['American', 'White Americans', 'American Express', 'African Americans', 'American Airlines', 'American Samoa', 'American football', 'American Idol', 'German Americans', 'American bison']
['Hartford City FC', 'Hartford City', 'Hartford, Connecticut', 'Hartford Athletic', 'National Premier Soccer League', 'Asa Hartford', 'Boston City FC', '2017 NPSL season', 'Al-Marzook Field at Alumni Stadium', 'New York City FC']
['University of Hartford', 'Hartford, Connecticut', 'West Hartford, Connecticut', 'Hartford Hawks', 'Univer

 ---------------------------------------------------------------------------- 

1192
Q:  What continents do the 31st parallel north and the 38th parallel north both cross?
A:  Asia and North America
The 38th parallel north is a circle of latitude that is 38 degrees north of the Earth's equatorial plane. It crosses Europe, the Mediterranean Sea, Asia, the Pacific Ocean, North America, and the Atlantic Ocean. The 38th parallel north formed the border between North and South Korea prior to the Korean War .
https://en.wikipedia.org/wiki/38th_parallel_north
In the United States, the 31st parallel defines part of the border between Mississippi and Louisiana, and part of the border between Alabama and Florida. The 31st parallel north is a circle of latitude that is 31 degrees north of the Earth's equatorial plane. It crosses Africa, Asia, the Pacific Ocean, North America and the Atlantic Ocean .
https://en.wikipedia.org/wiki/31st_parallel_north

Entities:
38th - 51 - 55 - ORDINAL - "first", "

['38th parallel north', '38th Street gang', '38th Street (Minneapolis)', '38th Infantry Division (United States)', 'Korean War', '38th Academy Awards', '38th Infantry Regiment (United States)', '38th Regiment', '38th Filmfare Awards', '38th SS-Grenadier-Division "Nibelungen"']
 ---------------------------------------------------------------------------- 



In [574]:
ids = random.sample(range(7921), 8)
for i in ids:
    k = str(i)
    Q = txt_dataset[k]['Q']
    print('Q: ', Q)
    for f in txt_dataset[k]['SupportingFacts']:
        print(f['url'])
        print(" ".join(f['url'].split('/')[-1].split("_")))
    doc = nlp(Q)
    if doc.ents: displacy.render(doc, style='ent', jupyter=True)
    else: print("No entities found")
    print(" ---------------------------------------------------------------------------- \n")

Q:  What commercially viable rock could you find near St., Thomas Mount?
https://en.wikipedia.org/wiki/St._Thomas_Mount
St. Thomas Mount
https://en.wikipedia.org/wiki/Stones_of_India
Stones of India


 ---------------------------------------------------------------------------- 

Q:  Which partner in the Byzantine–Venetian Treaty of 1082 had the statues of a pair of armored late Roman emperors embracing each other last?
https://en.wikipedia.org/wiki/Ionian_Islands_under_Venetian_rule
Ionian Islands under Venetian rule
https://en.wikipedia.org/wiki/Portrait_of_the_Four_Tetrarchs
Portrait of the Four Tetrarchs


 ---------------------------------------------------------------------------- 

Q:  What was the name of the deceased Desperate Housewives character that was friends with both the character who was originally planned to be interested in Orson Hodge and the character who ends up being interested in Orson?
https://en.wikipedia.org/wiki/Acquaintances_of_Gabrielle_Solis
Acquaintances of Gabrielle Solis
https://en.m.wikipedia.org/wiki/Paul_Young_(Desperate_Housewives)
Paul Young (Desperate Housewives)
https://en.wikipedia.org/wiki/Orson_Hodge_(Desperate_Housewives)
Orson Hodge (Desperate Housewives)


 ---------------------------------------------------------------------------- 

Q:  Were Cerdanya and Urgell still united when Wilfred was Count of Cerdanya?
https://en.wikipedia.org/wiki/County_of_Cerdanya
County of Cerdanya
https://en.wikipedia.org/wiki/Wifred_II,_Count_of_Cerdanya
Wifred II, Count of Cerdanya


 ---------------------------------------------------------------------------- 

Q:  Were there the same number of laps in both the 1961 and 1963 Syracuse Grand Prix?
https://en.wikipedia.org/wiki/1963_Syracuse_Grand_Prix
1963 Syracuse Grand Prix
https://en.wikipedia.org/wiki/1961_Syracuse_Grand_Prix
1961 Syracuse Grand Prix


 ---------------------------------------------------------------------------- 

Q:  A passage grave was a type of grave used primarily in which of the time periods in which Entrance graves were used?
https://en.wikipedia.org/wiki/Passage_grave
Passage grave
https://en.wikipedia.org/wiki/Entrance_grave
Entrance grave


 ---------------------------------------------------------------------------- 

Q:  How many years were there between the earliest known football match involving a women's French team and the suspension of the Ligue 1 season for COVID?
https://en.wikipedia.org/wiki/2019%E2%80%9320_Ligue_1
2019%E2%80%9320 Ligue 1
https://en.wikipedia.org/wiki/Women%27s_football_in_France
Women%27s football in France


 ---------------------------------------------------------------------------- 

Q:  Is the Accomodation reflex related to Horner's syndrome?
https://en.wikipedia.org/wiki/Horner%27s_syndrome
Horner%27s syndrome
https://en.wikipedia.org/wiki/Accommodation_reflex
Accommodation reflex


 ---------------------------------------------------------------------------- 



Cases where NER fails:
Which party controlled the **Senate** and the Presidency after the **1854** elections? But I need "1854 elections" / "1854 Senate"
On which body part are hoods and almuces worn? No entities found But I need "hoods" / "almuces"
Which caused more death; **Nazi Crimes** against the **Polish** nation or **Masterplan** album? But I need "Nazi Crimes against the Polist nation" / "Masterplan album"
What type of plant is both the Eucalyptus tereticornis and the **Corymbia** cadophora? But I need "Eucalyptus tereticornis" / "Corymbia cadophora"

In [19]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]

In [85]:
url_blocklist = ['seal', 'sign ', 'pdf', 'gif', 'icon', 'notice', 'cartoon', 'publish', 'menu', 'logo', 'svg', 'webm', 'page', \
                     'ogg', 'flickr', 'poster', 'ogv', 'banner', 'tif', 'montage', 'centralautologin', 'footer']

In [658]:
def get_page_categories(title):
    url = 'https://en.wikipedia.org/w/api.php?format=xml&action=query&prop=categories&titles='+title
    req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    with urllib.request.urlopen(req) as f:
        xml = f.read().decode('utf-8')
    soup= BeautifulSoup(xml,"lxml-xml")
    tags = soup.find('categories')
    if tags is None: return []
    categories = [c.get('title').replace("Category:", "") for c in tags]
    return categories


In [659]:
get_page_categories("Peter_Denton_(musician)")

[]

In [645]:
def is_disambiguation_page(url):
    title = url.split('/')[-1]
    return 'disambiguation' in " ".join(get_page_categories(title))

In [660]:
is_disambiguation_page("https://en.wikipedia.org/wiki/Peter_Denton_(musician)")

False

In [691]:
disambiguation_ks = []
effective = 0
count = 0
problematic_ks = []
for k in txt_dataset:
    if random.random() > 0.05: continue
    count += 1
    Q = txt_dataset[k]['Q']
    relevant_pages = set()
    try: 
        for f in txt_dataset[k]['SupportingFacts']:
            try: 
                fact_url = f['url']
                relevant_pages.add(fact_url)
                if is_disambiguation_page(fact_url):
                    disambiguation_ks.append(k)
                else:
                    relevant_pages = relevant_pages.union(find_hyperlinks(Q, fact_url))
            except (TypeError, AttributeError) as error: print(k, fact_url)
    except:
        print(k, "some error")
        problematic_ks.append(k)
    if len(relevant_pages) > len(txt_dataset[k]['SupportingFacts']):
        effective += 1
        print(k)
        pprint(relevant_pages)
print("count = ", count)
print(len(disambiguation_ks), len(set(disambiguation_ks)))
print(effective)

Q =  According to the French-inspired Arthurian romance tradition, who is the father of the character who was given a name by Chrétien involving a lake?
title =  Snake
Q =  According to the French-inspired Arthurian romance tradition, who is the father of the character who was given a name by Chrétien involving a lake?
title =  Snake
30
{'https://en.wikipedia.org/wiki/Lancelot',
 'https://en.wikipedia.org/wiki/Lancelot_of_the_Lake',
 'https://en.wikipedia.org/wiki/Snake'}
Q =  Which kind of tensor makes it possible to define several geometric notions on it and is present with a positive-definite metric tensor?
title =  Snake
Q =  Which kind of tensor makes it possible to define several geometric notions on it and is present with a positive-definite metric tensor?
title =  Snake
90
{'https://en.wikipedia.org/wiki/Metric_tensor',
 'https://en.wikipedia.org/wiki/Riemannian_metric',
 'https://en.wikipedia.org/wiki/Snake'}
Q =  Who was being addressed in the oldest known example of the lang

Q =  Were Command rank flags to denote the commander-in-chief of the English fleet used prior to or after the Rising Sun Ensign became a major symbol in the emerging Japanese Empire?
title =  Inca Empire
5192
{'https://en.wikipedia.org/wiki/Flag_of_Japan',
 'https://en.wikipedia.org/wiki/Inca_Empire',
 'https://en.wikipedia.org/wiki/List_of_command_flags_of_the_Royal_Navy'}
Q =  Were men, women or both burned at the stake as punishment for their crimes in the UK?
title =  Snake
Q =  Were men, women or both burned at the stake as punishment for their crimes in the UK?
title =  Snake
5368
{'https://en.wikipedia.org/wiki/Burning_of_women_in_England',
 'https://en.wikipedia.org/wiki/Snake',
 'https://en.wikipedia.org/wiki/Thomas_Aikenhead'}
Q =  In what year did both the Football League play-offs and then FIA Formula One World Championship take place?
title =  Snake
Q =  In what year did both the Football League play-offs and then FIA Formula One World Championship take place?
title =  Sna

Q =  The Metamorphoses of Apuleius and Amphitryon are both works that originated in what empire?
title =  Inca Empire
Q =  The Metamorphoses of Apuleius and Amphitryon are both works that originated in what empire?
title =  Inca Empire
7616
{'https://en.wikipedia.org/wiki/Amphitryon_(Plautus_play)',
 'https://en.wikipedia.org/wiki/Inca_Empire',
 'https://en.wikipedia.org/wiki/The_Golden_Ass'}
Q =  The Huma and  Amen Bird are both part of the mythology of what modern-day region?
title =  Inca mythology
Q =  The Huma and  Amen Bird are both part of the mythology of what modern-day region?
title =  Inca mythology
7630
{'https://en.wikipedia.org/wiki/Huma_bird',
 'https://en.wikipedia.org/wiki/Inca_mythology',
 'https://en.wikipedia.org/wiki/Iranian_folklore'}
Q =  What are three things thought to be required funerary practices by the people that thought that like Ra, their physical bodies, or Khat, would reawaken after they completed their journey through the underworld?
title =  Snake
Q 

In [677]:
tokenizer.tokenize('Incan')

['in', '##ca']

In [690]:
def find_hyperlinks(Q, fact_url):
    req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    with urllib.request.urlopen(req) as f:
        html = f.read().decode('utf-8')
    end_indx = html.find('<h2><span class="mw-headline" id="References">References</span>')
    html = html[:end_indx]
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', attrs={'href': re.compile("^/wiki/(?!.*(:|\(identifier\))).*")})
    links_stand_out = set()
    for link in links:
        title = link.get('title')
        text = link.text
        if title is None or text is None: continue
        if pylcs.lcs2(title.lower(), Q.lower())/len(title) > 0.5:
            links_stand_out.add('https://en.wikipedia.org' + link.get('href'))
            print("Q = ", Q)
            print("title = ", title)
        elif pylcs.lcs(title.lower(), text.lower())/len(text) < 0.85:
            if pylcs.lcs2(text.lower(), Q.lower())/len(text) > 0.5:
                print("Q = ", Q)
                print("text = ", text)
                links_stand_out.add('https://en.wikipedia.org' + link.get('href'))
    return links_stand_out

In [717]:
pos_list = ['NUM', 'NOUN', 'ADJ']
Q = 'What are three things thought to be required funerary practices by the people that thought that like Ra, their physical bodies, or Khat, would reawaken after they completed their journey through the underworld?'
keywords = [token.text for token in nlp(Q) if token.pos_ in pos_list or token.text[0].isupper()]
print(keywords)

['What', 'three', 'things', 'funerary', 'practices', 'people', 'Ra', 'physical', 'bodies', 'Khat', 'journey', 'underworld']


In [718]:
url = "https://en.wikipedia.org/wiki/Egyptian_tombs"
Q = 'What are three things thought to be required funerary practices by the people that thought that like Ra, their physical bodies, or Khat, would reawaken after they completed their journey through the underworld?'
req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
if True:
    with urllib.request.urlopen(req) as f:
        html = f.read().decode('utf-8')
    end_indx = html.find('<h2><span class="mw-headline" id="References">References</span>')
    html = html[:end_indx]
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', attrs={'href': re.compile("^/wiki/(?!.*(:|\(identifier\))).*")})
    links_stand_out = set()
    for link in links:
        title = link.get('title')
        text = link.text
        if title is None or text is None: continue
        #print(link)
        if any(b in keywords for b in title.split()):
            links_stand_out.add('https://en.wikipedia.org' + link.get('href'))
            print("title = ", title)
        elif pylcs.lcs(title.lower(), text.lower())/len(text) < 0.85:
            if any(b in keywords for b in text.split()):
                print("text = ", text)
                links_stand_out.add('https://en.wikipedia.org' + link.get('href'))
        #if pylcs.lcs2(title.lower(), Q.lower())/len(title) > 0.5:
            #links_stand_out.add('https://en.wikipedia.org' + link.get('href'))
            #print("title = ", title)
        #elif pylcs.lcs(title.lower(), text.lower())/len(text) < 0.85:
            #if pylcs.lcs2(text.lower(), Q.lower())/len(text) > 0.5:
                #print("text = ", text)
                #links_stand_out.add('https://en.wikipedia.org' + link.get('href'))
links_stand_out

title =  Cavern deities of the underworld
title =  Gate deities of the underworld
title =  Ra
title =  Eye of Ra
title =  Ancient Egyptian funerary texts
title =  Semitic people
title =  Ancient Egyptian funerary practices
title =  Ancient Egyptian funerary practices
title =  Ancient Egyptian funerary practices
title =  Ancient Egyptian funerary practices
title =  Ra
title =  Ancient Egyptian funerary texts
text =  funerary boat


{'https://en.wikipedia.org/wiki/Ancient_Egyptian_funerary_practices#cite_note-25',
 'https://en.wikipedia.org/wiki/Ancient_Egyptian_funerary_practices#cite_note-26',
 'https://en.wikipedia.org/wiki/Ancient_Egyptian_funerary_practices#cite_note-27',
 'https://en.wikipedia.org/wiki/Ancient_Egyptian_funerary_practices#cite_note-28',
 'https://en.wikipedia.org/wiki/Ancient_Egyptian_funerary_texts',
 'https://en.wikipedia.org/wiki/Cavern_deities_of_the_underworld',
 'https://en.wikipedia.org/wiki/Eye_of_Ra',
 'https://en.wikipedia.org/wiki/Gate_deities_of_the_underworld',
 'https://en.wikipedia.org/wiki/Khufu_ship',
 'https://en.wikipedia.org/wiki/Ra',
 'https://en.wikipedia.org/wiki/Semitic_people'}

In [705]:
wikipedia.summary("Ra", sentences=2, auto_suggest=False, redirect=True)

AttributeError: module 'wikipedia' has no attribute 'content'

In [708]:
wikipedia.page("Egyptian tombs", auto_suggest=False, redirect=True).content.split('\n')

['The ancient Egyptians had an elaborate set of funerary practices that they believed were necessary to ensure their immortality after death. These rituals included mummifying the body, casting magic spells, and burials with specific grave goods thought to be needed in the afterlife.The ancient Egyptian burial process evolved over time as old customs were discarded and new ones adopted, but several important elements of the process persisted. Although specific details changed over time, the preparation of the body, the magic rituals, and grave goods were all essential parts of a proper Egyptian funeral.',
 '',
 '',
 '== History ==',
 '',
 "Although no writing survived from the Predynastic Period in Egypt (c. 6000 – c. 3150 BCE), scholars believe the importance of the physical body and its preservation originated there. This likely explains why people of that time did not follow the common practice of cremation but rather buried the dead. Some also believe they may have feared the bodie

In [619]:
print(pylcs.lcs("Ionian School (music)", "Ionian School of music"))
print(len("Ionian School of music"))

19
22


In [599]:
### Scrap imgs and their captions. 
url = 'https://en.wikipedia.org/wiki/Forest_Red_Gum'
req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
with urllib.request.urlopen(req) as f:
    html = f.read().decode('utf-8')
end_indx = html.find('<h2><span class="mw-headline" id="References">References</span>')
html = html[:end_indx]
soup = BeautifulSoup(html, 'html.parser')
#links = soup.find_all('img', {'src':re.compile('.jpg|.png')})
links = soup.find_all('img')
print(len(links))
for l in links:
    imgUrl = l.get('src')
    #if "http" not in imgUrl: imgUrl = "https://" + url.split("/")[2] + imgUrl
    if int(l['width'])<100 or int(l['height'])<100:
        continue
    if any(b in imgUrl.lower() for b in url_blocklist): 
        continue
    print(" --------------------------------------------------------------- ")
    print(imgUrl)
    
    # Special case for thumb images, which are located inside a table
    thumbinner_div = l.find_parent("div", class_='thumbinner')
    if thumbinner_div: 
        print("parent thumbinner<div>: ", thumbinner_div.text)
        continue
        
    tr_parent = l.find_parent('tr')
    if tr_parent: print("parent<tr>: ", tr_parent.text)
        
    next_p = l.find_next('p')
    if next_p: print("<p>: ", next_p.text)
                
    prev_th = l.find_previous('th')
    if prev_th: print("prev<th>: ", prev_th.text)
    
    prev_p = l.find_previous('p')
    if prev_p: print("prev<p>: ", prev_p.text)


15
 --------------------------------------------------------------- 
//upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Eucalyptus_tereticornis_-_full_tree.jpg/220px-Eucalyptus_tereticornis_-_full_tree.jpg
parent<tr>:  


<p>:  Eucalyptus tereticornis, commonly known as forest red gum, blue gum or red irongum,[2] is a species of tree that is native to eastern Australia and southern New Guinea. It has smooth bark, lance-shaped to curved adult leaves, flower buds in groups of seven, nine or eleven, white flowers and hemispherical fruit.

prev<th>:  Forest red gum, blue gum, red irongum

prev<p>:  

 --------------------------------------------------------------- 
//upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Eucalyptus_terticornis_trees.jpg/220px-Eucalyptus_terticornis_trees.jpg
parent thumbinner<div>:   E. tereticornis trees


In [597]:
#html.find('<h2><span class="mw-headline" id="References">References</span>')
html[106916:106929]

'<h2><span cla'

In [489]:
### Given important spans, find relevant Wikipedia pages
import wikipedia
ny = wikipedia.search("Ferrocerium")
#print(ny.title)
#print(ny.url)
#print(ny.content)

In [490]:
ny

['Ferrocerium',
 'Flint',
 'Fire making',
 'List of named alloys',
 'Mischmetal',
 'Cerium',
 'Glossary of firelighting',
 'Lighter',
 'Neodymium magnet',
 'Tinder']